# Benchmarking the non-deep descriptors

Here we will evaluate all descriptors, which are available easily enough, e.g. from OpenCV or other python implementation. There will be final comparison table in the end of this notebook.

In [9]:
!pip install opencv-python --user
!pip install opencv-contrib-python --user 
!pip install kornia

# SIFT and RootSIFT, [kornia](https://kornia.readthedocs.io/en/latest/feature.html#kornia.feature.SIFTDescriptor) implementation

In [1]:
from IPython.display import clear_output
import torch
import kornia
from brown_phototour_revisited.benchmarking import *
def nice_results_string(desc_name, res_dict):
    if 'liberty' in res_dict:
        lib = f'{(100*res_dict["liberty"]):.2f}'
    else:
        lib = '-----'
    if 'notredame' in res_dict:
        notre = f'{(100*res_dict["notredame"]):.2f}'
    else:
        notre = '-----'
    if 'yosemite' in res_dict:
        yos = f'{(100*res_dict["yosemite"]):.2f}'
    else:
        yos = '-----'
    
    res = f'{desc_name[:20].ljust(20)}   {lib}      {notre}      {yos} '
    return res
TITLE = 'desc_name              liberty   notredame  yosemite'

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'
results_dir = 'data/mAP'

full_results_dict = {}

for patch_size in [32, 41, 65]:
    desc_name = 'Kornia RootSIFT'
    model = kornia.feature.SIFTDescriptor(patch_size, rootsift=True).eval()
    desc_dict = full_evaluation(model,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = patch_size, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()

In [2]:
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['3rdparty']))


desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 


In [3]:
for patch_size in [32, 41, 65]:
    desc_name = 'Kornia SIFT'
    model = kornia.feature.SIFTDescriptor(patch_size, rootsift=False).eval()
    desc_dict = full_evaluation(model,
                                'Kornia_SIFT',
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = patch_size, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['3rdparty']))

desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 
Kornia SIFT 32px       48.70      47.76      58.47 
Kornia SIFT 41px       48.30      47.30      58.14 
Kornia SIFT 65px       47.59      46.58      57.04 


# SIFT OpenCV

Now we will try OpenCV SIFT. It expects not patches, but full image and keypoints, so we will create a single keypoint in the center of the patch.

In [4]:
import cv2
import numpy as np
PS = 32
def get_center_kp(PS):
    c = PS/2.0
    center_kp = cv2.KeyPoint()
    center_kp.pt = (float(c),float(c))
    center_kp.size = PS/12.
    return center_kp

sift = cv2.SIFT_create()
def extract_opencv_sift(patch):
    return sift.compute((255*patch).astype(np.uint8),[get_center_kp(PS)])[1][0].reshape(128)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'


for PS in [32, 41, 65]:
    desc_name = 'OpenCV_SIFT'
    desc_dict = full_evaluation(extract_opencv_sift,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['3rdparty']))


desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 
Kornia SIFT 32px       48.70      47.76      58.47 
Kornia SIFT 41px       48.30      47.30      58.14 
Kornia SIFT 65px       47.59      46.58      57.04 
OpenCV_SIFT 32px       50.95      49.18      60.01 
OpenCV_SIFT 41px       53.20      50.81      63.61 
OpenCV_SIFT 65px       54.87      52.25      66.97 


What else do we have in OpenCV 4.4? 
- VGG (ConvexOpt). It is very cool descriptor, but there is no information about which subset provided descriptors were learned on :(
- LATCH (learned on yosemite)
- BoostDesc. Same as VGG
- DAISY. Same as VGG
- FREAK
- LUCID
- AKAZE

Let's go!

# [LATCH](https://arxiv.org/abs/1501.03719)

It is binary learned descriptor.

In [20]:
desc = cv2.xfeatures2d.LATCH_create()

def extract_latch(patch):
    return desc.compute((255*patch).astype(np.uint8),
                        [get_center_kp(PS)])[1][0].reshape(-1)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [65]: #32 crashes for LATCH.
    desc_name = 'OpenCV_LATCH'
    models = {"yosemite": extract_latch}
    desc_dict = full_evaluation(models,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()

In [6]:
print (TITLE)
for k, v in full_results_dict.items():
    if 'LATCH' in k:
        print (nice_results_string(k,  v['yosemite']))
    else:
        print (nice_results_string(k,  v['3rdparty']))
    

desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 
Kornia SIFT 32px       48.70      47.76      58.47 
Kornia SIFT 41px       48.30      47.30      58.14 
Kornia SIFT 65px       47.59      46.58      57.04 
OpenCV_SIFT 32px       50.95      49.18      60.01 
OpenCV_SIFT 41px       53.20      50.81      63.61 
OpenCV_SIFT 65px       54.87      52.25      66.97 
OpenCV_LATCH 65px      39.08      37.26      ----- 


# [LUCID](https://papers.nips.cc/paper/4706-locally-uniform-comparison-image-descriptor)

In [7]:
desc = cv2.xfeatures2d.LUCID_create()
def extract_lucid(patch):
    return desc.compute(cv2.cvtColor((255*patch).astype(np.uint8),cv2.COLOR_GRAY2RGB),
                        [get_center_kp(PS)])[1][0].reshape(-1)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [32, 65]:
    desc_name = 'OpenCV_LUCID'
    desc_dict = full_evaluation(extract_lucid,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    if 'LATCH' in k:
        print (nice_results_string(k,  v['yosemite']))
    else:
        print (nice_results_string(k,  v['3rdparty']))
    



desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 
Kornia SIFT 32px       48.70      47.76      58.47 
Kornia SIFT 41px       48.30      47.30      58.14 
Kornia SIFT 65px       47.59      46.58      57.04 
OpenCV_SIFT 32px       50.95      49.18      60.01 
OpenCV_SIFT 41px       53.20      50.81      63.61 
OpenCV_SIFT 65px       54.87      52.25      66.97 
OpenCV_LATCH 65px      39.08      37.26      ----- 
OpenCV_LUCID 32px      27.24      23.08      20.37 
OpenCV_LUCID 65px      27.65      23.31      20.53 


In [8]:
desc = cv2.xfeatures2d.FREAK_create()
def extract_freak(patch):
    return desc.compute((255*patch).astype(np.uint8),
                        [get_center_kp(PS)])[1][0].reshape(-1)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [65]:
    desc_name = 'OpenCV_FREAK'
    desc_dict = full_evaluation(extract_freak,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    if 'LATCH' in k:
        print (nice_results_string(k,  v['yosemite']))
    else:
        print (nice_results_string(k,  v['3rdparty']))

desc_name              liberty   notredame  yosemite
Kornia RootSIFT 32px   49.65      49.07      58.24 
Kornia RootSIFT 41px   49.01      48.48      57.83 
Kornia RootSIFT 65px   48.09      47.71      56.70 
Kornia SIFT 32px       48.70      47.76      58.47 
Kornia SIFT 41px       48.30      47.30      58.14 
Kornia SIFT 65px       47.59      46.58      57.04 
OpenCV_SIFT 32px       50.95      49.18      60.01 
OpenCV_SIFT 41px       53.20      50.81      63.61 
OpenCV_SIFT 65px       54.87      52.25      66.97 
OpenCV_LATCH 65px      39.08      37.26      ----- 
OpenCV_LUCID 32px      27.24      23.08      20.37 
OpenCV_LUCID 65px      27.65      23.31      20.53 
OpenCV_FREAK 65px      43.30      40.39      46.15 


In [29]:
TITLE1 = 'trained on       liberty notredame  liberty yosemite  notredame yosemite'
TITLE2 = 'tested  on           yosemite           notredame            liberty'


def nice_results_3rdparty(desc_name, res_dict):
    if 'liberty' in res_dict:
        lib = f'{(100*res_dict["liberty"]):.2f}'
    else:
        lib = '-----'
    if 'notredame' in res_dict:
        notre = f'{(100*res_dict["notredame"]):.2f}'
    else:
        notre = '-----'
    if 'yosemite' in res_dict:
        yos = f'{(100*res_dict["yosemite"]):.2f}'
    else:
        yos = '-----'
    res = f'{desc_name[:20].ljust(20)}   {yos}              {notre}               {lib} '
    return res

NA = '-----'
def nice_results_Brown(desc_name, res_dict):
    lib_yos, lib_notre, yos_notre, yos_lib, notre_lib, notre_yos = NA,NA,NA,NA,NA,NA
    if 'liberty' in res_dict:
        cr = res_dict['liberty']
        if 'notredame' in cr:
            lib_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            lib_notre = NA
        if 'yosemite' in cr:
            lib_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            lib_yos = NA
    if 'notredame' in res_dict:
        cr = res_dict['notredame']
        if 'liberty' in cr:
            notre_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            notre_lib = NA
        if 'yosemite' in cr:
            notre_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            notre_yos = NA    
    if 'yosemite' in res_dict:
        cr = res_dict['yosemite']
        if 'liberty' in cr:
            yos_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            yos_lib = NA
        if 'notredame' in cr:
            yos_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            yos_notre = NA    
    
    res = f'{desc_name[:20].ljust(18)} {lib_yos}  {lib_notre}        {notre_lib}  {notre_yos}        {yos_notre}  {yos_lib}'
    return res

def print_results_table(full_res_dict):
    print (TITLE1)
    print (TITLE2)

    for desc_name, desc_results in full_res_dict.items():
        if '3rdparty' in desc_results:
            #print (desc_results)
            if len(desc_results['3rdparty']) == 3:
                print (nice_results_3rdparty(desc_name, desc_results['3rdparty']))
            else:
                print (nice_results_Brown(desc_name, desc_results))
        else:
            print (nice_results_Brown(desc_name, desc_results))
    print ()
    return



In [26]:
print_results_table(full_results_dict)

trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia RootSIFT 65px   56.70              47.71               48.09 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
Kornia SIFT 65px       57.04              46.58               47.59 
OpenCV_SIFT 32px       60.01              49.18               50.95 
OpenCV_SIFT 41px       63.61              50.81               53.20 
OpenCV_SIFT 65px       66.97              52.25               54.87 
OpenCV_LATCH 65px  -----  -----        -----  -----        37.26  39.08
OpenCV_LUCID 32px      20.37              23.08               27.24 
OpenCV_LUCID 65px      20.53              23.31               27.65 
OpenCV_FREAK 65px      46.1

If you use the benchmark, please cite it:

    @misc{BrownRevisited2020,
      title={UBC PhotoTour Revisied},
      author={Mishkin, Dmytro},
      year={2020},
      url = {https://github.com/ducha-aiki/brown_phototour_revisited}
    }